In [23]:
!wget --no-check https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta
    
    


--2020-01-04 00:56:01--  https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 13.226.52.155, 13.226.52.168, 13.226.52.106, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|13.226.52.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 810105 (791K) [application/octet-stream]
Saving to: ‘chr1.GRCh38.excerpt.fasta’

chr1.GRCh38.excerpt 100%[===================>] 791.12K  --.-KB/s    in 0.1s    

2020-01-04 00:56:01 (6.14 MB/s) - ‘chr1.GRCh38.excerpt.fasta’ saved [810105/810105]



In [24]:
# readGenome
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            if not line[0] == '>':
                genome += line.rstrip()
    return genome
t = readGenome('chr1.GRCh38.excerpt.fasta')
t[:200]



'TTGAATGCTGAAATCAGCAGGTAATATATGATAATAGAGAAAGCTATCCCGAAGGTGCATAGGTCAACAATACTTGAGCCTAACTCAGTAGATCCTAAAAGAAAGCAATTTTTGCTGCTAACCTAACATTTCACAATGTCTGGAGACATTTACAGTTCCCACAACCTATGGCAGTTACTGGCATCTACTAGAGGTCAGAG'

In [25]:
def editDistApproMatch(p, t):
    # Create distance matrix
    D = []
    for i in range(len(p)+1):
        D.append([0]*(len(t)+1))
    # Initialize first row of matrix
    for i in range(len(p)+1):
        D[i][0] = i
    # Fill in the rest of the matrix
    for i in range(1, len(p)+1):
        for j in range(1, len(t)+1):
            delt = 0 if p[i-1] == t[j-1] else 1
            distHor = D[i][j-1] + 1
            distVer = D[i-1][j] + 1
            distDiag = D[i-1][j-1] + delt
            D[i][j] = min(distHor, distVer, distDiag)
    # Edit distance is the value in the bottom right corner of the matrix
    return min(D[len(p)])




In [26]:
%%time
# check
p = 'GCGTATGC'
t = 'TATTGGCTATACGGTT'
editDistApproMatch(p,t)



CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 125 µs


2

In [27]:
# Q1: What is the edit distance of the best match between pattern GCTGATCGATCGTACGGCTGATCGATCGTACG
# and the excerpt of human chromosome 1? (Don't consider reverse complements.)
p = 'GCTGATCGATCGTACG'
t = readGenome('chr1.GRCh38.excerpt.fasta')




In [28]:
%%time
editDistApproMatch(p,t)




CPU times: user 8.54 s, sys: 68 ms, total: 8.61 s
Wall time: 8.64 s


3

In [29]:
# Q2: What is the edit distance of the best match between pattern GCTGATCGATCGTACGGCTGATCGATCGTACG
# and the excerpt of human chromosome 1? (Don't consider reverse complements.)
p = 'GATTTACCAGATTGAG'
t = readGenome('chr1.GRCh38.excerpt.fasta')





In [30]:
%%time
editDistApproMatch(p,t)




CPU times: user 8.46 s, sys: 40 ms, total: 8.5 s
Wall time: 8.51 s


2

In [31]:
def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's prefix in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match
        
        
        


In [32]:
# download sequence reads
!wget --no-check https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.for_asm.fastq




--2020-01-04 00:56:18--  https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.for_asm.fastq
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 13.226.52.155, 13.226.52.168, 13.226.52.106, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|13.226.52.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2562951 (2.4M) [application/octet-stream]
Saving to: ‘ERR266411_1.for_asm.fastq’

ERR266411_1.for_asm 100%[===================>]   2.44M  14.8MB/s    in 0.2s    

2020-01-04 00:56:19 (14.8 MB/s) - ‘ERR266411_1.for_asm.fastq’ saved [2562951/2562951]



In [33]:
# Read the fastq sequence
def readFastq(filename):
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline()
            seq = fh.readline().rstrip()
            fh.readline()
            qual = fh.readline().rstrip()
            if len(seq) == 0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities




In [34]:
reads, quals = readFastq('ERR266411_1.for_asm.fastq')





In [35]:
reads[0]




'TAAACAAGCAGTAGTAATTCCTGCTTTATCAAGATAATTTTTCGACTCATCAGAAATATCCGAAAGTGTTAACTTCTGCGTCATGGAAGCGATAAAACTC'

In [36]:
# make index
import bisect
import sys
class Index(object):
    def __init__(self, t, k):
        ''' Create index from all substrings of size 'length' '''
        self.k = k  # k-mer length (k)
        self.index = []
        for i in range(len(t) - k + 1):  # for each k-mer
            self.index.append((t[i:i+k], i))  # add (k-mer, offset) pair
        self.index.sort()  # alphabetize by k-mer
    
    def query(self, p):
        ''' Return index hits for first k-mer of P '''
        kmer = p[:self.k]  # query with first k-mer
        i = bisect.bisect_left(self.index, (kmer, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != kmer:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits





In [37]:
kMers = {}
for i in range(len(reads)):
    index = Index(reads[i], k = 30)
    for mer in index.index:
        if mer[0] in kMers.keys():
            kMers[mer[0]] = kMers[mer[0]] | set([i])
        else:
            kMers[mer[0]] = set([i])
        
        
        


In [38]:
kMersSortedIndex = sorted(kMers.keys())



In [39]:
kMersSortedIndex[-10:]




['TTTTTTGACGAGTAACAAAGTTTGGATTGC',
 'TTTTTTGACGAGTAACAAAGTTTGGATTGT',
 'TTTTTTGACGAGTTACACAGTTTGGATTGC',
 'TTTTTTGAGTGGTGCACTTATTCAAGAAAA',
 'TTTTTTGATAAAGCAGGAATTACTACTGCT',
 'TTTTTTGCCTGTTTGGTTCGCTTTGAGGAG',
 'TTTTTTGGCGAGGGACAGAGGGTGGGTTGG',
 'TTTTTTTCATGGTGGAGTTTCTCCTGTTGA',
 'TTTTTTTGAGTGGTGCACTTATTCAAGAAA',
 'TTTTTTTGATAAAGCAGGAATTACTACTGC']

In [40]:
def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's prefix in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match
        





In [41]:
def kmer_overlap_map(reads, k):
    olaps = {}
    for b in range(len(reads)):
        if reads[b][:30] in kMers.keys():
            for a in kMers[reads[b][:30]]:
                if a == b:
                    continue
                olen = overlap(reads[a], reads[b], min_length=k)
                if olen > 0:
                    olaps[(a, b)] = olen
    return olaps





In [42]:
%%time
olaps = kmer_overlap_map(reads, k=30)




CPU times: user 5.41 s, sys: 116 ms, total: 5.52 s
Wall time: 5.53 s


In [44]:
len(olaps)




904746

In [45]:
keys = list(olaps.keys())




In [46]:
keys[:10]




[(2351, 2299),
 (8826, 7670),
 (4777, 5281),
 (9971, 9055),
 (6406, 7371),
 (3367, 4411),
 (2158, 2790),
 (4067, 2888),
 (4949, 4740),
 (9293, 8567)]

In [47]:
listNodeSuffixes = set()
for i in range(len(keys)):
    node = keys[i][0]
    listNodeSuffixes.add(node)





In [48]:
len(listNodeSuffixes)




7161